In [16]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import functions as func

file = "Jan-2017 onwards.csv"
original_df = pd.read_csv(file)

filter_col = ["month", "town", "flat_type", "storey_range", "floor_area_sqm", "remaining_lease", "resale_price"]
col_filtered_df = original_df[filter_col]


In [17]:
extracted_data = col_filtered_df['remaining_lease'].str.extract(
    r'(?P<years>\d+) years(?: (?P<months>\d+) months)?'
)

# Use .fillna(0) for any rows that might not match the pattern (coerced to NaN).
years = pd.to_numeric(extracted_data['years'], errors='coerce').fillna(0).astype(int)
months = pd.to_numeric(extracted_data['months'], errors='coerce').fillna(0).astype(int)

col_filtered_df.loc[:, 'lease_months'] = (years * 12) + months
col_filtered_df = col_filtered_df.drop(columns="remaining_lease")

C:\Users\wenyu\AppData\Local\Temp\ipykernel_21572\1226857134.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col_filtered_df.loc[:, 'lease_months'] = (years * 12) + months


In [18]:
amk_df = col_filtered_df[col_filtered_df["town"] == "ANG MO KIO"]
clem_df = col_filtered_df[col_filtered_df["town"] == "CLEMENTI"]
bb_df = col_filtered_df[col_filtered_df["town"] == "BUKIT BATOK"]


#####################################################
#2D Linear Regression Model#
Resale price over time
Categories: 
lease_months: 50-59yrs(600-719mths), 60-69yrs(720-839mths), 70-79yrs(840-959mths), 80-89yrs(960-1079mths), 90-99yrs(1080-1188mths) 
storey_range: 01 TO 03, 04 TO 06, 07 TO 09, 10 TO 12
flat_type: 3 ROOM, 4 ROOM, 5 ROOM

In [19]:
amk_1to3_50yrs_3rm = amk_df[(amk_df["lease_months"] >= 600) & 
                        (amk_df["lease_months"] < 720) & 
                        (amk_df["storey_range"] == "01 TO 03") &
                        (amk_df["flat_type"] == "3 ROOM")]

amk_1to3_50yrs_4rm = amk_df[(amk_df["lease_months"] >= 600) & 
                        (amk_df["lease_months"] < 720) & 
                        (amk_df["storey_range"] == "01 TO 03") &
                        (amk_df["flat_type"] == "4 ROOM")]

amk_1to3_50yrs_5rm = amk_df[(amk_df["lease_months"] >= 600) & 
                        (amk_df["lease_months"] < 720) & 
                        (amk_df["storey_range"] == "01 TO 03") &
                        (amk_df["flat_type"] == "5 ROOM")]

all_4to6_50yrs_3rm = col_filtered_df[(col_filtered_df["lease_months"] >= 600) & 
                                     (col_filtered_df["lease_months"] < 720) & 
                                     (col_filtered_df["storey_range"] == "04 TO 06") &
                                     (col_filtered_df["flat_type"] == "3 ROOM")]
# all_4to6_50yrs_3rm

In [20]:
#Filter outliers
q1 = all_4to6_50yrs_3rm['resale_price'].quantile(0.25)
q3 = all_4to6_50yrs_3rm['resale_price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
all_4to6_50yrs_3rm = all_4to6_50yrs_3rm[(all_4to6_50yrs_3rm['resale_price'] > lower_bound) & (all_4to6_50yrs_3rm['resale_price'] < upper_bound)]
# all_4to6_50yrs_3rm

In [21]:
#Train-test split
all_4to6_50yrs_3rm['month'] = pd.to_datetime(all_4to6_50yrs_3rm['month'])

# 2. Filter using the year property
# Part 1: 2017 to 2021
training_data_df = all_4to6_50yrs_3rm[(all_4to6_50yrs_3rm['month'].dt.year >= 2017) & (all_4to6_50yrs_3rm['month'].dt.year <= 2021)]

# Part 2: 2022 to 2025
testing_data_df = all_4to6_50yrs_3rm[(all_4to6_50yrs_3rm['month'].dt.year >= 2022) & (all_4to6_50yrs_3rm['month'].dt.year <= 2025)]


In [ ]:
#Feature engineer training month column (with normalisation to prevent Exploding Gradient)
start_date = training_data_df['month'].min()
training_data_df['days_passed'] = (training_data_df['month'] - start_date).dt.days
min_date = training_data_df['days_passed'].min()
max_date = training_data_df['days_passed'].max()
training_data_df['days_passed_scaled'] = (training_data_df['days_passed'] - min_date) / max_date - min_date
# training_data_df

#Feature engineer testing month column
start_date = testing_data_df['month'].min()
testing_data_df['days_passed'] = (testing_data_df['month'] - start_date).dt.days
min_date = testing_data_df['days_passed'].min()
max_date = testing_data_df['days_passed'].max()
testing_data_df['days_passed_scaled'] = (testing_data_df['days_passed'] - min_date) / max_date - min_date

In [ ]:
#Convert training df to np.array
feature = ["days_passed_scaled", "resale_price"]
training_data_df = training_data_df[feature]
training_data_np = pd.DataFrame(training_data_df).to_numpy()
# training_data_np

# Convert testing df to np.array
features = ['days_passed_scaled', 'resale_price']
testing_data_np = pd.DataFrame(testing_data_df[features]).to_numpy()
# testing_data_np
# for i in testing_data_np:
#     test_x, test_y = i
#     print(test_x)


In [25]:
# Run Batch Gradient Descent to find optimal Weight vector
trained_weight = func.batchGradientDescent(func.mseLoss, func.lossGradient, len(feature), training_data_np, func.linearPhi)

# Validate model
total_error = 0
for i in testing_data_np:
    test_x, test_y = i
    pred_y = trained_weight.dot(func.linearPhi(test_x))
    squared_error = (pred_y - test_y) ** 2
    total_error += squared_error
mse = 1 / len(testing_data_np) * total_error
print("MSE: ", mse)

epoch: 0 Cost: 90663605053.03063965 Loss_gradient: [-329268.39659753 -592921.9924465 ] Weight: [32926.83965975 59292.19924465]
epoch: 10 Cost: 3611339444.01099586 Loss_gradient: [ -4380.80657637 -34021.35220591] Weight: [111629.4128598  221931.66105024]
epoch: 20 Cost: 3263555208.90585566 Loss_gradient: [ 9433.64656593 -7223.2867149 ] Weight: [105078.78205551 235483.04043388]
epoch: 30 Cost: 3139716560.64797115 Loss_gradient: [ 8919.32405397 -5297.94909027] Weight: [ 95784.98943374 241373.34547191]
epoch: 40 Cost: 3044157251.58040953 Loss_gradient: [ 7876.76202218 -4612.30717491] Weight: [ 87443.16476341 246275.13880287]
epoch: 50 Cost: 2970176844.95205069 Loss_gradient: [ 6931.95418177 -4055.98683363] Weight: [ 80095.69707883 250575.04781791]
epoch: 60 Cost: 2912901932.88216352 Loss_gradient: [ 6099.35622037 -3568.6783018 ] Weight: [ 73630.4444498  254357.84602683]
epoch: 70 Cost: 2868560251.25533724 Loss_gradient: [ 5366.70962761 -3140.00671147] Weight: [ 67941.77588689 257686.229339

In [ ]:
# TODO: Cross-Validation
# TODO: Quadratic Regression
# TODO: Periodic Regression